In [44]:
import tensorflow as tf
from tensorflow import keras as K

In [45]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [46]:
 tf.config.list_physical_devices('GPU')

[]

In [47]:
## Create Dataset with tensorflow dataset:

In keras we have preprocessing that helps nicely how to collect train and valid sets:

In [50]:
home = '/home/ddpham/practice/swift_swallow'
# home = '/path/to/your/project'

In [51]:
bs = 32
train_ds = K.preprocessing.image_dataset_from_directory(
    f'{home}/data/train/'
    , labels='inferred'
    , label_mode='categorical'
    , batch_size=bs
    , image_size=(224, 224)
)

valid_ds = K.preprocessing.image_dataset_from_directory(
    f'{home}/data/valid/'
    , labels='inferred'
    , label_mode='categorical'
    , batch_size=bs
    , image_size=(224, 224)
)

Found 714 files belonging to 2 classes.
Found 203 files belonging to 2 classes.


In [52]:
model = K.applications.ResNet50(include_top=False, weights="imagenet", input_shape=[224, 224, 3])

In [53]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [54]:
flatten = K.layers.Flatten()(model.output)
predictions = tf.keras.layers.Dense(2, activation='sigmoid', name="predictions")(flatten)
new_model = K.models.Model(inputs=model.input, outputs=predictions)
new_model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [63]:
auc = K.metrics.AUC()
new_model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=['accuracy', auc])

In [62]:
new_model.fit(train_ds, epochs=10, validation_data=valid_ds)

Epoch 1/10
23/23 [==============================] - 110s 5s/step - loss: 3.2286 - accuracy: 0.5896 - auc: 0.5528 - val_loss: 28730222592.0000 - val_accuracy: 0.4433 - val_auc: 0.4433
Epoch 2/10
23/23 [==============================] - 109s 5s/step - loss: 2.3563 - accuracy: 0.6078 - auc: 0.6431 - val_loss: 4631334.0000 - val_accuracy: 0.4433 - val_auc: 0.5000
Epoch 3/10
23/23 [==============================] - 109s 5s/step - loss: 1.6088 - accuracy: 0.6162 - auc: 0.6725 - val_loss: 13652.9189 - val_accuracy: 0.4384 - val_auc: 0.4384
Epoch 4/10
23/23 [==============================] - 109s 5s/step - loss: 1.2819 - accuracy: 0.6148 - auc: 0.6605 - val_loss: 69213.8984 - val_accuracy: 0.5567 - val_auc: 0.5567
Epoch 5/10
23/23 [==============================] - 109s 5s/step - loss: 1.7461 - accuracy: 0.5938 - auc: 0.6176 - val_loss: 57258.3359 - val_accuracy: 0.5567 - val_auc: 0.5513
Epoch 6/10
23/23 [==============================] - 109s 5s/step - loss: 1.1398 - accuracy: 0.6387 - auc: 0

We can see that using the same pretrained model with some default options on optimizer, learning rate from keras, we have results that are far less good compared to fastai. This is not because Keras is worst than fastai, but my point here is that fastai has custom the learning schduler, the training methodology that gain best results for each pretrained model, we dont need to care much about it. Of course, if we want to understand why & how the model work, it's always worth digging down further to understand the whole process, however, for newbies (like me) it'll take more time and effor to do it, we can learn more by reading documents & source code.

To do better with keras, there're a number of things we need to do:
1. Working on the data (augmentations and stuff, this hasn't been done here)
2. Working on learning rate scheduler
3. Along with 2 is finding the right learning rate to start with.

Let's do that step by step to see if we can do the same with Keras (i'm sure experts would do the same, but we -amatuers- can try to understand better how the whole things work).